In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer,GenerationConfig,Trainer,TrainingArguments,DataCollatorForSeq2Seq
import time
import pandas as pd
import numpy as np
from datasets import Dataset

In [3]:
# 设置设备参数
DEVICE = "cuda"  # 使用CUDA
DEVICE_ID = "0"  # CUDA设备ID，如果未设置则为空
CUDA_DEVICE = f"{DEVICE}:{DEVICE_ID}" if DEVICE_ID else DEVICE  # 组合CUDA设备信息

def torch_gc():
    if torch.cuda.is_available():  # 检查是否可用CUDA
        torch.cuda.empty_cache()  # 清空CUDA缓存
        torch.cuda.ipc_collect()  # 收集CUDA内存碎片
torch_gc()

In [4]:
df = pd.read_json('./datasets/huanhuan.json')
ds = Dataset.from_pandas(df)
ds[:6]

{'instruction': ['小姐，别的秀女都在求中选，唯有咱们小姐想被撂牌子，菩萨一定记得真真儿的——',
  '这个温太医啊，也是古怪，谁不知太医不得皇命不能为皇族以外的人请脉诊病，他倒好，十天半月便往咱们府里跑。',
  '嬛妹妹，刚刚我去府上请脉，听甄伯母说你来这里进香了。',
  '嬛妹妹，我虽是一介御医，俸禄微薄，可是我保证会一生一世对你好，疼爱你，保护你，永远事事以你为重。本来没半月一次到府上去请脉，能够偶尔见一次妹妹的笑靥，已经心满意足了，可谁知——而且我也知道，妹妹心里是不愿意去殿选的。',
  '实初虽然唐突了妹妹，却是真心实意地希望妹妹不要去应选，这不仅仅是因为我心里一直把妹妹当成……其实更是因为甄伯父曾经救过家父的性命。',
  '可是我父亲当年被诬，起因也是因为后宫争斗，不能独善其身。一介御医尚且如此，何况妹妹如果被选中的话，会身在其中啊。'],
 'input': ['', '', '', '', '', ''],
 'output': ['嘘——都说许愿说破是不灵的。',
  '你们俩话太多了，我该和温太医要一剂药，好好治治你们。',
  '出来走走，也是散心。',
  '实初哥哥这么说，就枉顾我们一直以来的兄妹情谊了，嬛儿没有哥哥，一直把你当作自己的亲哥哥一样看待，自然相信哥哥会待妹妹好的——自然了，以后有了嫂子，你也会对嫂子更好。',
  '我们两家是世交，昔年恩义不过是父亲随手之劳，不必挂怀。',
  '实初哥哥的话我都明白，只是我不去应选，迟早也是玉娆，家中无子，女儿还能不孝吗？'],
 '__index_level_0__': [0, 1, 2, 3, 4, 5]}

In [6]:
tokenizer = AutoTokenizer.from_pretrained("./models/qwen/Qwen2___5-7B-Instruct",use_fast=True,trust_remote_code=True)
tokenizer

Qwen2TokenizerFast(name_or_path='./models/qwen/Qwen2___5-7B-Instruct', vocab_size=151643, model_max_length=131072, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '<|im_end|>', 'pad_token': '<|endoftext|>', 'additional_special_tokens': ['<|im_start|>', '<|im_end|>', '<|object_ref_start|>', '<|object_ref_end|>', '<|box_start|>', '<|box_end|>', '<|quad_start|>', '<|quad_end|>', '<|vision_start|>', '<|vision_end|>', '<|vision_pad|>', '<|image_pad|>', '<|video_pad|>']}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	151643: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151644: AddedToken("<|im_start|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151645: AddedToken("<|im_end|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151646: AddedToken("<|object_ref_start|>", rstrip=False, lstrip=False, single_

In [7]:
#lora训练的数据
def process_func(example):
    MAX_LENGTH = 384    # Llama分词器会将一个中文字切分为多个token，因此需要放开一些最大长度，保证数据的完整性
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer(f"<|im_start|>system\n现在你要扮演皇帝身边的女人--甄嬛<|im_end|>\n<|im_start|>user\n{example['instruction'] + example['input']}<|im_end|>\n<|im_start|>assistant\n", add_special_tokens=False)  # add_special_tokens 不在开头加 special_tokens
    response = tokenizer(f"{example['output']}", add_special_tokens=False)
    #输入是input和output拼接
    input_ids = instruction["input_ids"] + response["input_ids"] + [tokenizer.pad_token_id]
    #attention_mask掩码位置 对应input_ids 区分填充和真实token
    attention_mask = instruction["attention_mask"] + response["attention_mask"] + [1]  # 因为eos token咱们也是要关注的所以 补充为1
    #labels是input_ids的token的id，但是填充的token的id是-100，这个是lora的默认配置 需要的是数据中output
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"] + [tokenizer.pad_token_id]  
    if len(input_ids) > MAX_LENGTH:  # 做一个截断
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }
    

In [8]:
tokenizer_id = ds.map(process_func,remove_columns=ds.column_names)

Map:   0%|          | 0/3729 [00:00<?, ? examples/s]

In [9]:
tokenizer.decode(tokenizer_id[0]['input_ids'])

'<|im_start|>system\n现在你要扮演皇帝身边的女人--甄嬛<|im_end|>\n<|im_start|>user\n小姐，别的秀女都在求中选，唯有咱们小姐想被撂牌子，菩萨一定记得真真儿的——<|im_end|>\n<|im_start|>assistant\n嘘——都说许愿说破是不灵的。<|endoftext|>'

In [10]:
tokenizer.decode(list(filter(lambda x:x!=-100,tokenizer_id[1]['labels'])))

'你们俩话太多了，我该和温太医要一剂药，好好治治你们。<|endoftext|>'

In [12]:
torch_gc()
model = AutoModelForCausalLM.from_pretrained("./models/qwen/Qwen2___5-7B-Instruct",device_map='auto',torch_dtype=torch.bfloat16)

/usr/local/lib/python3.9/site-packages/accelerate/utils/modeling.py:1462: UserWarning: Current model requires 469765632 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.


In [13]:
model.enable_input_require_grads()

In [14]:
model.dtype

torch.bfloat16

### lora

In [15]:
from peft import LoraConfig, TaskType, get_peft_model,PeftModel

config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, 
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    inference_mode=False, # 训练模式
    r=8, # Lora 秩
    lora_alpha=32, # Lora alaph，具体作用参见 Lora 原理
    lora_dropout=0.1# Dropout 比例
)
config


LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules={'o_proj', 'down_proj', 'up_proj', 'v_proj', 'k_proj', 'q_proj', 'gate_proj'}, lora_alpha=32, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False))

In [16]:
model = get_peft_model(model,config)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [17]:
model.print_trainable_parameters()

trainable params: 20,185,088 || all params: 7,635,801,600 || trainable%: 0.2643


In [18]:
args = TrainingArguments(
    output_dir="./outputs/Qwen2.5_instruct_lora",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    logging_steps=10,
    num_train_epochs=3,
    save_steps=100, 
    learning_rate=1e-4,
    save_on_each_node=True,
    gradient_checkpointing=True
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenizer_id,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
)

NotImplementedError: Cannot copy out of meta tensor; no data!

In [ ]:
trainer.train()

In [ ]:
mode_path = './models/qwen/Qwen2___5-7B-Instruct'
lora_path = './outputs/Qwen2.5_instruct_lora'

In [ ]:
# 加载tokenizer
tokenizer = AutoTokenizer.from_pretrained(mode_path, trust_remote_code=True)

# 加载模型
model = AutoModelForCausalLM.from_pretrained(mode_path, device_map="auto",torch_dtype=torch.bfloat16, trust_remote_code=True).eval()
model = PeftModel(model,lora_path=lora_path)

prompt = "你谁啊？"
inputs = tokenizer.apply_chat_template([{"role":"system","content": "假设你是皇帝身边的女人--甄嬛。"},{"role":"user","content":prompt}], 
                                    return_tensors="pt",
                                    add_generation_prompt=True,
                                    tokenize=True,
                                    return_dict=True,).to('cuda')
gen_kwargs = {"max_length": 2500, "do_sample": True, "top_k": 1}
with torch.no_grad():
    outputs = model.generate(**inputs, **gen_kwargs)
    outputs = outputs[:,inputs['input_ids'].shape[1]:]
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))